In [12]:
n = 8
board = [['.' for i in range(n)] for i in range(n)]

In [13]:
for x in range(2):
    for y in range(8):
        board[x][y] = "W"      

In [14]:
print("\n".join(' '.join(c for c in row) for row in board))

W W W W W W W W
W W W W W W W W
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .


In [15]:
for x in range(6, 8):
    for y in range(8):
        board[x][y] = "B"   

In [16]:
print("\n".join(' '.join(c for c in row) for row in board))

W W W W W W W W
W W W W W W W W
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
B B B B B B B B
B B B B B B B B


In [17]:
board = [['.' for i in range(n)] for i in range(n)]

for i in range(2):
    for j in range(8):
        board[i][j] = 'B'

for i in range(6, 8):
    for j in range(8):
        board[i][j] = 'W'

def moves(to_move):
    list_moves = set()
    for i in range(0,8):
        for j in range(0,8):
            if board[i][j] == 'B' and to_move == 2 and j+1 <= 7:
                if board[i+1][j] == '.' and i+1 <= 7:
                    list_moves.add("{pos1}-{pos2}".format(pos1=pos_convert(i,j), pos2=pos_convert(i+1,j)))
                if (board[i+1][j+1] == 'W' or board[i+1][j+1] == '.') and i+1 <= 7 and j+1 <= 7:
                    list_moves.add("{pos1}-{pos2}".format(pos1=pos_convert(i,j), pos2=pos_convert(i+1,j+1)))
                if (board[i+1][j-1] == 'W' or board[i+1][j-1] == '.') and i+1 <=7 and j-1 >= 0:
                    list_moves.add("{pos1}-{pos2}".format(pos1=pos_convert(i,j), pos2=pos_convert(i+1,j-1)))
            if board[i][j] == 'B' and to_move == 2 and j+1 == 8:
                if board[i+1][j] == '.' and i+1 <= 7:
                    list_moves.add("{pos1}-{pos2}".format(pos1=pos_convert(i,j), pos2=pos_convert(i+1,j)))
                if (board[i+1][j-1] == 'W' or board[i+1][j-1] == '.') and i+1 <= 7:
                    list_moves.add("{pos1}-{pos2}".format(pos1=pos_convert(i,j), pos2=pos_convert(i+1,j-1)))           

            if board[i][j] == 'W' and to_move == 1 and j+1 <= 7:
                if board[i-1][j] == '.' and i-1 >= 0:
                    list_moves.add("{pos1}-{pos2}".format(pos1=pos_convert(i,j), pos2=pos_convert(i-1,j)))
                if (board[i-1][j+1] == 'B' or board[i-1][j+1] == '.') and i-1 >= 0 and j+1 <= 7:
                    list_moves.add("{pos1}-{pos2}".format(pos1=pos_convert(i,j), pos2=pos_convert(i-1,j+1)))
                if (board[i-1][j-1] == 'B' or board[i-1][j-1] == '.') and i-1 >= 0 and j-1 >= 0:
                    list_moves.add("{pos1}-{pos2}".format(pos1=pos_convert(i,j), pos2=pos_convert(i-1,j-1)))
            if board[i][j] == 'W' and to_move == 1 and j+1 == 8:
                if board[i-1][j] == '.' and i-1 >= 0:
                    list_moves.add("{pos1}-{pos2}".format(pos1=pos_convert(i,j), pos2=pos_convert(i-1,j)))
                if (board[i-1][j-1] == 'B' or board[i-1][j-1] == '.') and i-1 >= 0:
                    list_moves.add("{pos1}-{pos2}".format(pos1=pos_convert(i,j), pos2=pos_convert(i-1,j-1)))
        
    return list_moves

def pos_convert(i, j):
    alfabeto = "abcdefgh"
    return alfabeto[j] + str(i + 1)

In [24]:
move = "h2"
numbers = []
alfabeto = "abcdefgh"
numbers.append(int(alfabeto.rfind(move[0])))
numbers.append(int(move[1]) - 1)
print(numbers)

[7, 1]


In [27]:
board = [['.' for i in range(n)] for i in range(n)]
print(len(board))

8
